# Exploratory Data Analysis of B-Cell Data

In this notebook we features and structure of B-cell data from the Dana Pe'er Lab of Computational Systems Biology. As part of their experiments, Pe’er et.al collected a B cell centric, mass cytometry dataset from human bone marrow. Their dataset includes records of multiple B cell cellular features, such as activation of regulatory sig-naling molecules, and phenotypic proteins.  We will be using their publicly available data of mass cytometry and scRNA-seq. 

For more information about the Dana Pe'er Lab and their Wanderlust algorithm: https://dpeerlab.github.io/dpeerlab-website/wanderlust.html

Their publicly available B cell data: https://dpeerlab.github.io/dpeerlab-website/wanderlust-data.html

## Analyzing Flow Cytometry Data

In [ ]:
# data management
import numpy as np
import pandas as pd

# flow cytometry libraries
import FlowCytometryTools
import FlowCal